# Limpieza informacion usuario

Este código limpia los datos que tiene Giganav de sus usuarios y los distribuye para ser utilizado para llamadas.

Importo repositorios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform

Establezco directorios

In [2]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")
    folder=os.getcwd()
    data=folder+'\\'+'Datos'
    facturas=data+'\\'+'Facturas'
    tickets=data+'\\'+'Tickets'
    usuarios=data+'\\'+'Usuarios'
    egresos=data+'\\'+'Egresos'
    transacciones=data+'\\'+'Transacciones'
    analisis=folder+'\\'+'Operacion\\Analisis'
    bases=analisis+'\\'+'Bases_analisis'
    outputs=analisis+'\\'+'Output'
    plots=outputs+'\\'+'Gráficos'
    usuarios_clean=bases+'\\'+'Usuarios'
    transac_clean=bases+'\\'+'Transacciones'
    tickets_clean=bases+'\\'+'Tickets'
    facturas_clean=bases+'\\'+'Facturas'
    egresos_clean=bases+'\\'+'Egresos'
    
if platform.system()=='Darwin':
    #Directorios Mac
    folder='/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV'
    data=folder+'/Datos'
    facturas=data+'/Facturas'
    tickets=data+'/Tickets'
    usuarios=data+'/Usuarios'
    egresos=data+'/Egresos'
    transacciones=data+'/Transacciones'
    analisis=folder+'/Operacion/Analisis'
    bases=analisis+'/Bases_analisis'
    outputs=analisis+'/Outputs'
    plots=outputs+'/Gráficos'
    usuarios_clean=bases+'/Usuarios'
    transac_clean=bases+'/Transacciones'
    tickets_clean=bases+'/Tickets'
    facturas_clean=bases+'/Facturas'
    egresos_clean=bases+'/Egresos'

## OJO debe coincidir con la fecha de descarga de los archivos

In [3]:
fecha=pd.to_datetime('today').strftime('%y_%m_%d')
fecha

'24_10_08'

Importo datos de usuarios

In [4]:
import glob
list_of_files = glob.glob(os.path.join(usuarios,'Lista de Usuarios*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_usuarios=pd.read_csv(latest_file, low_memory=False)
base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           0 non-null      float64
 1   Cedula               3891 non-null   object 
 2   Id                   3893 non-null   int64  
 3   Mac                  113 non-null    object 
 4   Nombre               3893 non-null   object 
 5   Instalado            2281 non-null   object 
 6   Dirección Principal  3866 non-null   object 
 7   Deuda actual         1361 non-null   object 
 8   Plan voip            1327 non-null   object 
 9   Plan                 2281 non-null   object 
 10  Ip                   2281 non-null   object 
 11  Tipo estrato         3893 non-null   int64  
 12  Fecha suspendido     3893 non-null   object 
 13  Ultimo pago          3816 non-null   object 
 14  Correo               2958 non-null   object 
 15  Telefono             2919 non-null   o

Me quedo con información relevante

In [5]:
depurada_base_usuarios=base_usuarios.loc[:,[
    'Id', 'Nombre', 'Plan voip', 
    'Plan', 'Deuda actual', 'Tipo estrato', 
    'Correo', 'Telefono', 'Movil', 
    'Zona', 'Total cobrar']]
depurada_base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            3893 non-null   int64 
 1   Nombre        3893 non-null   object
 2   Plan voip     1327 non-null   object
 3   Plan          2281 non-null   object
 4   Deuda actual  1361 non-null   object
 5   Tipo estrato  3893 non-null   int64 
 6   Correo        2958 non-null   object
 7   Telefono      2919 non-null   object
 8   Movil         3824 non-null   object
 9   Zona          3866 non-null   object
 10  Total cobrar  3893 non-null   object
dtypes: int64(2), object(9)
memory usage: 334.7+ KB


In [6]:
depurada_base_usuarios.head()

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar
0,3376,Prueba Prueba RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
1,3378,PRUEBA GIGANAV RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,VILLA FANNY,"COL$ 0,00"
2,1374,RODRIGO SILVA SEPULVEDA ACTIVO,NaN,GIGABASIC RURAL,NaN,1,silva_rodrigo_25@hotmail.com,3206671131,3153598524,CGTO MINAS,"COL$ 120.000,00"
3,2450,PUNTO GRATIS ACTIVO,NaN,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
4,788,JHON FREDY FONSECA CARRANZA RETIRADO,NaN,NaN,"3 COL$ 182.000,00",1,jhonfredyfonsecacarranza@gmail.com,3162662078,3187613025,CGTO LA LLANA,"COL$ 0,00"


Creo categorias de usuarios y limpio nombre

In [7]:
categorias_usuarios=['activo', 'retirado', 'suspendido']

for j, i in enumerate(categorias_usuarios):
    #Creo estado del usuario
    depurada_base_usuarios.loc[
        depurada_base_usuarios['Nombre'].str.contains(i, case=False), 
        'estado']=categorias_usuarios[j]
    
#Creo nombre corregido
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'Nombre'].str.replace('activo', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('retirado', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('suspendido', '', case=False)

Creo variable de TV

In [8]:
#Lleno Nan
depurada_base_usuarios.loc[:,'Plan voip'].fillna('', inplace=True)

#Creo variable
depurada_base_usuarios.loc[:, 'TV']=depurada_base_usuarios[
    'Plan voip'].str.contains('recaudo a terceros', case=False)*1

depurada_base_usuarios['TV'].value_counts()

/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_2624/3975338928.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  depurada_base_usuarios.loc[:,'Plan voip'].fillna('', inplace=True)


TV
0    2566
1    1327
Name: count, dtype: int64

Creo variable de megas

In [9]:
#Creo megas que maneja la empresa
megas=[10, 15,25, 30, 40, 50, 60, 100, 150]

#Limpio Nan de variable plan
depurada_base_usuarios.loc[:, 'Plan'].fillna('', inplace=True)

#Creo variable megas
depurada_base_usuarios.loc[:,'megas']=np.select([
    depurada_base_usuarios['Plan'].str.contains('10X10', case=False), 
    depurada_base_usuarios['Plan'].str.contains('15X15', case=False),  
    depurada_base_usuarios['Plan'].str.contains('25X25', case=False), 
    depurada_base_usuarios['Plan'].str.contains('30X30', case=False), 
    depurada_base_usuarios['Plan'].str.contains('40X40', case=False), 
    depurada_base_usuarios['Plan'].str.contains('50X50', case=False), 
    depurada_base_usuarios['Plan'].str.contains('60X60', case=False), 
    depurada_base_usuarios['Plan'].str.contains('100X100|100X30', case=False), 
    depurada_base_usuarios['Plan'].str.contains('150X150', case=False),], 
    choicelist=megas)
#Limpio vacíos
depurada_base_usuarios.loc[
    depurada_base_usuarios['megas']=='0','megas']=0

depurada_base_usuarios['megas'].value_counts().sort_values(ascending=False)

/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_2624/2924166334.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  depurada_base_usuarios.loc[:, 'Plan'].fillna('', inplace=True)


megas
0      2792
50      598
10      152
15       98
60       72
100      68
30       47
25       42
40       22
150       2
Name: count, dtype: int64

Visualizo número de usuarios por segmento de la empresa

In [10]:
mask_megas=depurada_base_usuarios['megas']!=0

pd.pivot_table(
    depurada_base_usuarios.loc[mask_megas], index='megas', columns='TV', values='Id', aggfunc='nunique')

TV,0,1
megas,,
10,52,100
15,20,78
25,14,28
30,19,28
40,4,18
50,209,389
60,22,50
100,40,28
150,1,1


Creo variable radio

In [11]:
depurada_base_usuarios.loc[:, 'radio']=depurada_base_usuarios[
    'Plan'].str.contains('giga', case=False)*1

depurada_base_usuarios['radio'].value_counts()

radio
0    2713
1    1180
Name: count, dtype: int64

In [12]:
#Creo total a cobrar
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['Total cobrar'].str.strip('COL$ ')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].str.replace('.','')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].str.replace(',','.')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].astype('float')
depurada_base_usuarios.loc[
    depurada_base_usuarios['TV']==1,'tarifa']=depurada_base_usuarios['tarifa']+25000

In [13]:
depurada_base_usuarios.loc[depurada_base_usuarios['estado']=='activo']

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar,estado,nombre,TV,megas,radio,tarifa
2,1374,RODRIGO SILVA SEPULVEDA ACTIVO,,GIGABASIC RURAL,NaN,1,silva_rodrigo_25@hotmail.com,3206671131,3153598524,CGTO MINAS,"COL$ 120.000,00",activo,RODRIGO SILVA SEPULVEDA,0,0,1,120000.0
3,2450,PUNTO GRATIS ACTIVO,,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00",activo,PUNTO GRATIS,0,30,0,0.0
6,1722,MARIA ALEJANDRA RESTREPO MORALES ACTIVO,,GIGA VIP PLUS 100M SIMETRICAS,NaN,1,Marymkery124@gmail.com,3128227880,3105378643,LOS PINOS,"COL$ 80.000,00",activo,MARIA ALEJANDRA RESTREPO MORALES,0,0,1,80000.0
7,3806,2-MARIA ALEJANDRA RESTREPO MORALES ACTIVO,,GIGA VIP PLUS 100M SIMETRICAS,NaN,1,marymkery124@gmail.com,3123140321,3105378643,1º DE MAYO,"COL$ 70.000,00",activo,2-MARIA ALEJANDRA RESTREPO MORALES,0,0,1,70000.0
10,4176,LUIS ALFREDO MEZA MONTIEL ACTIVO,,50X50X150 GPON3,NaN,1,luisalfredomezamontiel1987@gmail.com,3166472810,3185066065,CGTO LA LLANA,"COL$ 50.000,00",activo,LUIS ALFREDO MEZA MONTIEL,0,50,0,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3882,2380,2-FREDY PEREZ MORA ACTIVO,,50X50X150 GPON2,NaN,1,fredyperezmora1982@gmail.com,3188868005,3172972735,CGTO LA PEDREGOSA,"COL$ 50.000,00",activo,2-FREDY PEREZ MORA,0,50,0,50000.0
3886,3879,JOSE LUIS PEREZ MONTERO ACTIVO,,60X60X60 GPON 3,NaN,1,jperez.jlpm24@gmail.com,3122140668,3107291306,SAN RAFAEL,"COL$ 60.000,00",activo,JOSE LUIS PEREZ MONTERO,0,60,0,60000.0
3888,21,ODAIR MORA VALBUENA ACTIVO,,GIGA VIP 100M SIMETRICAS,NaN,2,odairmoravalbuena@gmail.com,NaN,3152180728,NUEVO HORIZONTE,"COL$ 52.000,00",activo,ODAIR MORA VALBUENA,0,0,1,52000.0
3890,1038,JORGE ALBERTO MORENO ACTIVO,,50X50X150 GPON2,NaN,1,NaN,3142082576,3188925978,CGTO LA PEDREGOSA,"COL$ 52.000,00",activo,JORGE ALBERTO MORENO,0,50,0,52000.0


In [14]:
variables=['Id', 'nombre', 'estado', 
           'radio', 'Tipo estrato', 'megas', 
           'TV', 'Telefono', 'Movil', 
           'Correo', 'Zona', 'tarifa', 
           'Deuda actual']
base_final_contactos=depurada_base_usuarios.loc[:,variables]
base_final_contactos

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,Deuda actual
0,3376,Prueba Prueba,retirado,0,1,0,0,NaN,NaN,NaN,NaN,0.0,NaN
1,3378,PRUEBA GIGANAV,retirado,0,1,0,0,NaN,NaN,NaN,VILLA FANNY,0.0,NaN
2,1374,RODRIGO SILVA SEPULVEDA,activo,1,1,0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,NaN
3,2450,PUNTO GRATIS,activo,0,1,30,0,NaN,NaN,NaN,NaN,0.0,NaN
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,0,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,"3 COL$ 182.000,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,21,ODAIR MORA VALBUENA,activo,1,2,0,0,NaN,3152180728,odairmoravalbuena@gmail.com,NUEVO HORIZONTE,52000.0,NaN
3889,2875,ALIRIO DE JESUS MONSALVE OSPINA,retirado,0,1,0,0,3126429826,3213262547,aliriooodejesusmonsalve1805@gmail.com,CGTO LA PEDREGOSA,0.0,"1 COL$ 25.000,00"
3890,1038,JORGE ALBERTO MORENO,activo,0,1,50,0,3142082576,3188925978,NaN,CGTO LA PEDREGOSA,52000.0,NaN
3891,3265,JESUS MARIA HERNANDEZ HENAO,retirado,0,1,0,1,3145291879,3208370958,jesusmariahernan@gmail.com,CGTO LA PEDREGOSA,25000.0,NaN


In [15]:
#Leo base Hubspot y agrego record ID para actualizar contactos existentes

In [16]:
list_of_files = glob.glob(os.path.join(usuarios,'Usuarios_export_Hubspot*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_hubspot=pd.read_csv(latest_file, low_memory=False)
base_hubspot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3956 entries, 0 to 3955
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID de registro           3956 non-null   int64  
 1   Nombre                   3950 non-null   object 
 2   Apellidos                17 non-null     object 
 3   estado suscripción       3924 non-null   object 
 4   Tarifa                   3924 non-null   float64
 5   Megas                    3924 non-null   float64
 6   TV                       3924 non-null   float64
 7   Telefono                 2973 non-null   object 
 8   Movil                    3858 non-null   object 
 9   Correo de miembro        2951 non-null   object 
 10  Última actividad         502 non-null    object 
 11  ID_mikrowisp             3924 non-null   float64
 12  Zona                     3897 non-null   object 
 13  Etapa del ciclo de vida  3956 non-null   object 
 14  meses_suspendido        

In [17]:
#Cambio nombre y hago merge para pegar solo el Record ID de Hubspot
#Elimino duplicados y me quedo con el record de hubspot mas reciente
base_hubspot.rename(columns={
    'ID_mikrowisp': 'Id'}, inplace=True)

base_hubspot=base_hubspot.groupby(
        'Id').agg(
        id_hubspot=(
        'ID de registro', 'max')).reset_index()

base_hubspot.rename(columns={
    'id_hubspot': 'ID de registro'}, inplace=True)

#Hago merge
base_records_hubspot=base_final_contactos.merge(base_hubspot.loc[
    :,['Id', 'ID de registro']], on='Id', how='left')

base_records_hubspot.loc[
    :, 'ID de registro']=base_records_hubspot[
    'ID de registro'].astype('float')

base_records_hubspot.head()

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,Deuda actual,ID de registro
0,3376,Prueba Prueba,retirado,0,1,0,0,NaN,NaN,NaN,NaN,0.0,NaN,21911.0
1,3378,PRUEBA GIGANAV,retirado,0,1,0,0,NaN,NaN,NaN,VILLA FANNY,0.0,NaN,20907.0
2,1374,RODRIGO SILVA SEPULVEDA,activo,1,1,0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,NaN,25003.0
3,2450,PUNTO GRATIS,activo,0,1,30,0,NaN,NaN,NaN,NaN,0.0,NaN,35701.0
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,0,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,"3 COL$ 182.000,00",25405.0


In [18]:
base_records_hubspot.loc[base_records_hubspot['estado']=='suspendido','Deuda actual']#.str.split(
   # ' ', expand=True, n=1)

5       1 COL$ 72.000,00
8       1 COL$ 13.000,00
13      1 COL$ 67.000,00
23      1 COL$ 77.000,00
24      1 COL$ 52.000,00
              ...       
3868    1 COL$ 54.000,00
3871    1 COL$ 82.000,00
3884    1 COL$ 67.000,00
3885    1 COL$ 75.000,00
3887    1 COL$ 72.000,00
Name: Deuda actual, Length: 320, dtype: object

In [19]:
base_records_hubspot.loc[
    :,'meses_suspendido']= base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido',
    'Deuda actual'].str.split(
    ' ', expand=True, n=1).iloc[:,0].fillna(0).astype(
    'int')

base_records_hubspot.loc[
    :,'deuda']= base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido',
    'Deuda actual'].str.split(
    ' ', expand=True, n=1).iloc[:,1].str.strip(
    'COL$').str.replace(
    '.','').str.replace(
    ',00','').str.lstrip().str.replace(',','.').fillna(0).astype('float')



In [20]:
print('El número de usuarios suspendidos al '+fecha+' es '+str(base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido', 'Id'].groupby(level=0).nunique().sum()))

El número de usuarios suspendidos al 24_10_08 es 320


Exporto base para hubspot

In [21]:
filepath=os.path.join(usuarios_clean,'Base_usuarios_hub_'+fecha)
base_records_hubspot.to_csv(filepath+'.csv', index=False)

##### 